# BDH Model Training on Google Colab with Google Drive

This notebook trains the BDH model on novels from Google Drive and uses train.csv for labels.

**Steps:**
1. Mount Google Drive
2. Load files from Drive
3. Train the model
4. Download the trained model


In [ ]:
# Install dependencies
!pip install torch numpy tqdm google-api-python-client google-auth-httplib2 google-auth-oauthlib -q
print("✓ Dependencies installed")


## Step 1: Mount Google Drive


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
print("✓ Google Drive mounted")

# Set your Google Drive folder path here
# Example: '/content/drive/MyDrive/KDSH26' or '/content/drive/MyDrive/Books'
GDRIVE_FOLDER_PATH = '/content/drive/MyDrive/KDSH26'  # ⬅️ CHANGE THIS to your folder path

print(f"\n📁 Using folder: {GDRIVE_FOLDER_PATH}")
print("   Make sure this folder contains:")
print("   - Your novel .txt files")
print("   - train.csv")
print("   - test.csv (optional)")


## Step 2: Upload Core Files

Upload these Python files from your local machine:
- `core/bdh.py`
- `utils/bdh_narrative_builder.py`


In [ ]:
from google.colab import files
import os

# Create directory structure
os.makedirs('core', exist_ok=True)
os.makedirs('utils', exist_ok=True)
os.makedirs('Books', exist_ok=True)

print("📁 Please upload these Python files:")
print("1. bdh.py (from core/bdh.py)")
print("2. bdh_narrative_builder.py (from utils/bdh_narrative_builder.py)")
print("\n⚠️  IMPORTANT: Click 'Choose Files' and select BOTH files at once!")
print("   Hold Ctrl (Windows) or Cmd (Mac) to select multiple files\n")

# Upload files
uploaded = files.upload()

print(f"\n📦 Uploaded {len(uploaded)} file(s):")
for filename in uploaded.keys():
    print(f"   - {filename}")

# Organize uploaded files
print("\n📂 Organizing files...")
moved_bdh = False
moved_builder = False

for filename in uploaded.keys():
    # Handle bdh.py (could be named bdh.py or core/bdh.py)
    if filename == 'bdh.py' or filename.endswith('/bdh.py') or 'bdh.py' in filename:
        target_path = 'core/bdh.py'
        if os.path.exists(filename):
            os.rename(filename, target_path)
            print(f"✓ Moved {filename} → {target_path}")
            moved_bdh = True
    # Handle bdh_narrative_builder.py
    elif 'bdh_narrative_builder.py' in filename:
        target_path = 'utils/bdh_narrative_builder.py'
        if os.path.exists(filename):
            os.rename(filename, target_path)
            print(f"✓ Moved {filename} → {target_path}")
            moved_builder = True

# Check if files are in place
print("\n✅ Verification:")
if os.path.exists('core/bdh.py'):
    print("   ✓ core/bdh.py exists")
else:
    print("   ✗ core/bdh.py MISSING - please upload bdh.py")

if os.path.exists('utils/bdh_narrative_builder.py'):
    print("   ✓ utils/bdh_narrative_builder.py exists")
else:
    print("   ✗ utils/bdh_narrative_builder.py MISSING - please upload bdh_narrative_builder.py")

if moved_bdh and moved_builder:
    print("\n🎉 Both files uploaded successfully!")
else:
    print("\n⚠️  Some files are missing. Please run this cell again and upload both files.")


## Step 3: Load Books and CSV from Google Drive


In [ ]:
import sys
import os
from pathlib import Path
import shutil

# Add paths
sys.path.insert(0, 'core')
sys.path.insert(0, 'utils')

# Copy books from Google Drive to local Books folder
print("📚 Loading books from Google Drive...")
books = []
book_files = list(Path(GDRIVE_FOLDER_PATH).glob("*.txt"))

if not book_files:
    print(f"⚠️  No .txt files found in {GDRIVE_FOLDER_PATH}")
    print("   Please check the folder path above!")
else:
    for book_file in book_files:
        print(f"  Loading {book_file.name}...")
        # Copy to local Books folder
        shutil.copy(book_file, f'Books/{book_file.name}')
        # Read content
        with open(book_file, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
            books.append(text)
            print(f"    ✓ Loaded {len(text):,} characters")

print(f"\n✅ Loaded {len(books)} novels")

# Load train.csv from Google Drive
print("\n📋 Loading train.csv from Google Drive...")
train_csv_path = Path(GDRIVE_FOLDER_PATH) / "train.csv"
if train_csv_path.exists():
    shutil.copy(train_csv_path, 'train.csv')
    print("✓ train.csv loaded")
else:
    print("⚠️  train.csv not found in Google Drive folder")

# Load test.csv (optional)
test_csv_path = Path(GDRIVE_FOLDER_PATH) / "test.csv"
if test_csv_path.exists():
    shutil.copy(test_csv_path, 'test.csv')
    print("✓ test.csv loaded")
else:
    print("⚠️  test.csv not found (optional)")


In [ ]:
import torch
import numpy as np
from tqdm import tqdm

from bdh import BDH, BDHConfig
from bdh_narrative_builder import _text_to_tokens

print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


## Step 5: Train Model

This uses **MEMORY-EFFICIENT** training code to prevent crashes:
- ✅ Mixed precision training - **1.5-2x faster + less memory**
- ✅ `torch.compile()` - **2-3x faster** (optional, can disable if crashes)
- ✅ Random batches (no pre-allocation)
- ✅ CUDA optimizations
- ✅ Memory cleanup every 500 steps

**Training time: 20-35 minutes** on Colab GPU

**Settings (memory-safe):**
- `max_iters=3000` iterations
- `batch_size=16` (reduced to prevent OOM)
- `block_size=256` (reduced to prevent OOM)
- Token limit: 1M (prevents crashes)

**If session crashes:**
1. Reduce `batch_size=8` in the code below
2. Reduce `block_size=128`
3. Set `USE_COMPILE = False` in the function
4. Reduce `max_iters=2000` for faster completion


In [ ]:
# Copyright Pathway Technology, Inc.
# Adapted to use books from Google Drive instead of Shakespeare dataset

import os
from contextlib import nullcontext
import bdh
import numpy as np
import tempfile
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dtype = (
    "bfloat16"
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else "float16"
)  # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler

ptdtype = {
    "float32": torch.float32,
    "bfloat16": torch.bfloat16,
    "float16": torch.float16,
}[dtype]

ctx = (
    torch.amp.autocast(device_type=device.type, dtype=ptdtype)
    if "cuda" in device.type
    else nullcontext()
)

scaler = torch.amp.GradScaler(device=device.type, enabled=(dtype == "float16"))

torch.manual_seed(1337)
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn

print(f"Using device: {device} with dtype {dtype}")

# Configuration
BDH_CONFIG = bdh.BDHConfig()
BLOCK_SIZE = 512
BATCH_SIZE = 16  # Reduced from 32 to prevent OOM crashes
MAX_ITERS = 3000
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0.1
LOG_FREQ = 100

# If you still get OOM errors, reduce these:
# BATCH_SIZE = 8
# BLOCK_SIZE = 256

# Prepare input file from books (instead of fetch_data())
print("Preparing training data from books...")
combined_text = "\n\n".join(books)
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.txt')
temp_file.write(combined_text.encode('utf-8'))
temp_file.close()
input_file_path = temp_file.name
print(f"Training data prepared: {len(combined_text):,} characters")

def get_batch(split):
    # treat the file as bytes
    data = np.memmap(input_file_path, dtype=np.uint8, mode="r")
    if split == "train":
        data = data[: int(0.9 * len(data))]
    else:
        data = data[int(0.9 * len(data)) :]
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack(
        [torch.from_numpy((data[i : i + BLOCK_SIZE]).astype(np.int64)) for i in ix]
    )
    y = torch.stack(
        [
            torch.from_numpy((data[i + 1 : i + 1 + BLOCK_SIZE]).astype(np.int64))
            for i in ix
        ]
    )
    if torch.cuda.is_available():
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(
            device, non_blocking=True
        )
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# Training (EXACTLY like original)
model = bdh.BDH(BDH_CONFIG).to(device)

# torch.compile can cause crashes on some Colab GPUs - make it optional
try:
    print("Compiling model (this may take 1-2 minutes)...")
    model = torch.compile(model)
    print("✓ Model compiled successfully")
except Exception as e:
    print(f"⚠️  Compilation failed: {e}")
    print("Continuing without compilation (slower but stable)")

optimizer = torch.optim.AdamW(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)

x, y = get_batch("train")
loss_acc = 0
loss_steps = 0

try:
    for step in range(MAX_ITERS):
        with ctx:
            logits, loss = model(x, y)
        x, y = get_batch("train")
        loss_acc += loss
        loss_steps += 1
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if step % LOG_FREQ == 0:
            print(f"Step: {step}/{MAX_ITERS} loss {loss_acc.item() / loss_steps:.3}")
            loss_acc = 0
            loss_steps = 0
        
        # Periodic memory cleanup
        if step % 500 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\n❌ GPU Out of Memory!")
        print("Solutions:")
        print("1. Reduce BATCH_SIZE to 8")
        print("2. Reduce BLOCK_SIZE to 256")
        print("3. Use Runtime > Change runtime type > GPU > T4 (if available)")
        raise
    else:
        raise

print("Training done, now generating a sample ")
model.eval()
prompt = torch.tensor(
    bytearray("To be or ", "utf-8"), dtype=torch.long, device=device
).unsqueeze(0)
ret = model.generate(prompt, max_new_tokens=100, top_k=3)
ret_decoded = bytes(ret.to(torch.uint8).to("cpu").squeeze(0)).decode(
    errors="backslashreplace"
)
print(ret_decoded)

# Cleanup temp file
try:
    os.unlink(input_file_path)
except:
    pass


In [ ]:
os.makedirs('models', exist_ok=True)

model_path = 'models/bdh_trained.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'epochs': 50
}, model_path)

print(f"✅ Model saved to {model_path}")
print(f"   File size: {os.path.getsize(model_path) / 1e6:.1f} MB")


## Step 7: Download Model

Download the trained model to your local machine.


In [ ]:
from google.colab import files

print("📥 Downloading trained model...")
files.download(model_path)
print("\n✅ Download complete!")
print("\n📋 Next steps:")
print("1. Move the downloaded file to: Main_System/scripts/models/bdh_trained.pt")
print("2. Run: cd Main_System/scripts && python3 bdh_quickstart.py")
print("3. The script will automatically use the trained model!")
